# OpenAI 텍스트 생성(Text Generation) API 살펴보기

## API Reference : https://platform.openai.com/docs/api-reference/chat/create

In [ ]:
!pip install openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
!pip show openai

Name: openai
Version: 1.6.1
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: llmx


# 1. seed 파라미터를 이용해서 재현가능한 텍스트 생성 결과 만들기

## Reference : https://cookbook.openai.com/examples/deterministic_outputs_with_the_seed_parameter

## OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

In [ ]:
import asyncio
import pprint
import difflib
from IPython.display import display, HTML

GPT_MODEL = "gpt-3.5-turbo-1106"

In [ ]:
# 이 함수는 GPT로부터 응답 결과를 받아옵니다.
async def get_chat_response(system_message: str, user_request: str, seed: int = None):
    try:
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_request},
        ]

        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            seed=seed,
            max_tokens=200,
            temperature=0.7,
        )

        response_content = response.choices[0].message.content
        system_fingerprint = response.system_fingerprint
        prompt_tokens = response.usage.prompt_tokens
        completion_tokens = (
            response.usage.total_tokens - response.usage.prompt_tokens
        )

        table = f"""
        <table>
        <tr><th>Response</th><td>{response_content}</td></tr>
        <tr><th>System Fingerprint</th><td>{system_fingerprint}</td></tr>
        <tr><th>Number of prompt tokens</th><td>{prompt_tokens}</td></tr>
        <tr><th>Number of completion tokens</th><td>{completion_tokens}</td></tr>
        </table>
        """
        display(HTML(table))

        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# 이 함수는 두 응답을 비교하고 차이점을 표로 표시합니다.
# 삭제된 부분은 빨간색으로, 추가된 부분은 초록색으로 강조됩니다.
# 차이점이 없는 경우 "차이점이 발견되지 않았습니다."라고 출력합니다.
def compare_responses(previous_response: str, response: str):
    d = difflib.Differ()
    diff = d.compare(previous_response.splitlines(), response.splitlines())

    diff_table = "<table>"
    diff_exists = False

    for line in diff:
        if line.startswith("- "):
            diff_table += f"<tr style='color: red;'><td>{line}</td></tr>"
            diff_exists = True
        elif line.startswith("+ "):
            diff_table += f"<tr style='color: green;'><td>{line}</td></tr>"
            diff_exists = True
        else:
            diff_table += f"<tr><td>{line}</td></tr>"

    diff_table += "</table>"

    if diff_exists:
        display(HTML(diff_table))
    else:
        print("No differences found.")

## seed 파라미터 설정 없이 비교하기

In [ ]:
topic = "가을"
system_message = "You are a helpful assistant that generates short poems."
user_request = f"{topic}에 관한 짧은 시를 작성해줘."

previous_response = await get_chat_response(
    system_message=system_message, user_request=user_request
)

response = await get_chat_response(
    system_message=system_message, user_request=user_request
)

# 이 함수는 두 응답을 비교하고 차이점을 표로 표시합니다.
# 삭제된 부분은 빨간색으로, 추가된 부분은 초록색으로 강조됩니다.
# 차이점이 없는 경우 "차이점이 발견되지 않았습니다."라고 출력합니다.
compare_responses(previous_response, response)

Response,가을이 오면 단풍이 물들고 바람은 시원해지고 가을은 참 아름다워
System Fingerprint,fp_772e8125bb
Number of prompt tokens,37
Number of completion tokens,39


Response,가을이 왔네 단풍잎이 떨어지네 하늘은 맑고 바람은 시원하고 가을은 아름답네
System Fingerprint,fp_772e8125bb
Number of prompt tokens,37
Number of completion tokens,51


- 가을이 오면
- 단풍이 물들고
+ 가을이 왔네
+ 단풍잎이 떨어지네
+ 하늘은 맑고
- 바람은 시원해지고
? ^^
+ 바람은 시원하고
? ^
- 가을은 참 아름다워
+ 가을은 아름답네


## seed 파라미터 설정후 비교하기

In [ ]:
SEED = 123
response = await get_chat_response(
    system_message=system_message, seed=SEED, user_request=user_request
)
previous_response = response
response = await get_chat_response(
    system_message=system_message, seed=SEED, user_request=user_request
)

compare_responses(previous_response, response)

Response,산 너머 갈대숲에 바람 소리 가을이 오는 소식을 전해주네 단풍이 노랗게 물들고 가을의 멋진 향기가 퍼져간다
System Fingerprint,fp_772e8125bb
Number of prompt tokens,37
Number of completion tokens,70


Response,산 너머 갈대숲에 바람 소리 가을이 오는 소식을 전해주네 단풍이 노랗게 물들고 가을의 멋진 향기가 퍼져간다.
System Fingerprint,fp_772e8125bb
Number of prompt tokens,37
Number of completion tokens,71


산 너머 갈대숲에 바람 소리
가을이 오는 소식을 전해주네
단풍이 노랗게 물들고
- 가을의 멋진 향기가 퍼져간다
+ 가을의 멋진 향기가 퍼져간다.
? +


# 2. 토큰 개수 세기

## Reference : https://platform.openai.com/docs/guides/text-generation/managing-tokens

In [ ]:
import tiktoken

In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
      See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")

In [ ]:
messages = [
  {"role": "system", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
  {"role": "system", "name":"example_user", "content": "New synergies will help drive top-line growth."},
  {"role": "system", "name": "example_assistant", "content": "Things working well together will increase revenue."},
  {"role": "system", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
  {"role": "system", "name": "example_assistant", "content": "Let's talk later when we're less busy about how to do better."},
  {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
]

model = "gpt-3.5-turbo-0613"

print(f"{num_tokens_from_messages(messages, model)} prompt tokens counted.")
# Should show ~126 total_tokens

126 prompt tokens counted.


In [ ]:
# example token count from the OpenAI API
response = client.chat.completions.create(
  model=model,
  messages=messages,
  temperature=0,
)

print(f'{response.usage.prompt_tokens} prompt tokens used.')

129 prompt tokens used.


# 3. JSON mode 사용해보기

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020?"}
  ]
)
print(response.choices[0].message.content)

{
  "winner": "Los Angeles Dodgers"
}


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "너는 JSON 출력을 만드는 조수야. 아래 내용의 감정상태가 '긍정'인지 '부정'인지 판단해줘"},
    {"role": "user", "content": "가격이 착하고 디자인이 예쁩니다"}
  ]
)
print(response.choices[0].message.content)

{
  "감정상태": "긍정"
}


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "너는 JSON 출력을 만드는 조수야. 아래 내용의 감정상태가 '긍정'인지 '부정'인지 판단해줘"},
    {"role": "user", "content": "금액이 저렴해도 기대를 했었는데 그 값어치밖에 안되는군요 그만큼만 입겠습니다"}
  ]
)
print(response.choices[0].message.content)

{
  "감정상태": "부정"
}


# 4. Temperature 값 변경해보기

## temperature : 설정가능범위(0.0~2.0, 기본값 1.0), 낮을수록 더 정확한 답변, 높을수록 더 다양성 있는 답변을 생성

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "너는 도움이 되는 조수야."},
    {"role": "user", "content": "푸른 하늘에 OO이 떠있다. OO에 들어갈 단어를 10개 추천해줘"}
  ]
)
print(response.choices[0].message.content)

불꽃, 구름, 비행기, 이끼, 기억, 별, 햇볕, 구름, 비행기, 비둘기


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "너는 도움이 되는 조수야."},
    {"role": "user", "content": "푸른 하늘에 OO이 떠있다. OO에 들어갈 단어를 10개 추천해줘"}
  ],
  temperature=0.0
)
print(response.choices[0].message.content)

1. 구름
2. 태양
3. 달
4. 비행기
5. 구름
6. 비
7. 별
8. 비둘기
9. 구름
10. 무지개


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "너는 도움이 되는 조수야."},
    {"role": "user", "content": "푸른 하늘에 OO이 떠있다. OO에 들어갈 단어를 10개 추천해줘"}
  ],
  temperature=2.0
)
print(response.choices[0].message.content)

달, 비행기, 구름, 별, 태양, 비, 월, 구름, 비행 로켓, 핫물 갓Teacherarezendi"?>
MayorAPI AudioImage


# 5. frequency_penalty & presence_penalty 값 변경해보기

**presence_penalty** : -2.0과 2.0 사이의 숫자입니다. (기본값 0.0) 양수 값은 지금까지의 텍스트에 나타나는 **새로운 토큰을 기반으로 벌칙을 부여**하여 모델이 새로운 주제에 대해 이야기할 가능성을 높입니다.

**frequency_penalty** : -2.0과 2.0 사이의 숫자입니다. (기본값 0.0) 양수 값은 현재까지 텍스트에서의 **존재 빈도를 기준으로 새 토큰에 대해 벌칙을 부여**하며, 모델이 동일한 문장을 그대로 반복할 가능성을 감소시킵니다.






In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "너는 도움이 되는 조수야."},
    {"role": "user", "content": "가을 하면 생각나는 단어를 20개 추천해줘"}
  ]
)
print(response.choices[0].message.content)

1. 단풍
2. 가을 열대과일
3. 호박
4. 바람
5. 가을 축제
6. 단호박
7. 감귤
8. 비올 때
9. 꽃
10. 따뜻한 차
11. 나무
12. 달콤한 사과
13. 단맛
14. 포근한 담요
15. 가을 햇볕
16. 단단한 마룬
17. 달 보름달
18. 버섯
19. 단콩
20. 메론


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "너는 도움이 되는 조수야."},
    {"role": "user", "content": "가을 하면 생각나는 단어를 20개 추천해줘"}
  ],
  presence_penalty=-2.0
)
print(response.choices[0].message.content)

물들기, 단풍, 시원한 바람, 담배 연기, 단단한 껍질, 햇볕, 바람, 담배, 햇볕, 단풍, 나뭇잎, 나뭇가지, 느끼한 공기, 고요한 밤, 시들은 꽃, 땀, 담배 연기, 시들은 잎, 바람소리, 단풍잎


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "너는 도움이 되는 조수야."},
    {"role": "user", "content": "가을 하면 생각나는 단어를 20개 추천해줘"}
  ],
  presence_penalty=2.0
)
print(response.choices[0].message.content)

좋아, 여기 가을을 상징하는 단어 20가지야:

1. 단풍
2. 감자 수확
3. 나무 잎
4. 가을비
5. 케이크
6. 향긋한 차
7. 바람
8. 달
9. 현미경
10. 가을 축제
11. 개구리 소리
12. 호박
13. 사과 농장
14. 단단한 호두
15. 고산윗독
16. 나귀
17. 커피
18. 버터스카치
19. 벙커 계덧망
20. 읽는 시간

이 추천이 도움이 되었기를 바라며, 다른 것이 필요하면 언제든 말해줘!


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "너는 도움이 되는 조수야."},
    {"role": "user", "content": "가을 하면 생각나는 단어를 20개 추천해줘"}
  ],
  frequency_penalty=-2.0
)
print(response.choices[0].message.content)

좋아요, 여기 몇 가지 가을을 상징하는 단어들이에요.
1. 단풍
2. 가을 단풍
3. 가을풍경
4. 가을풍
5. 가을 가을
6. 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가을 가


In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "너는 도움이 되는 조수야."},
    {"role": "user", "content": "가을 하면 생각나는 단어를 20개 추천해줘"}
  ],
  frequency_penalty=2.0
)
print(response.choices[0].message.content)

좋아요, 여기 가을과 연관된 20가지 단어에 대한 추천입니다:

1. 단풍
2. 감자
3. 산소
4. 청명
5. 열매
6. 시원한 바람 
7. 걷기 
8. 학교 
9 . 왁스 베리 그래나딘 ,
10 . 수박,
11 . 병충해 관리,
12 . 참외,
13 . 슬레이트 빛깔 ,
14 굴뚝 ,   
15 미 서터니ть ,   
16 펄잡운공주의 이야기     
17 악세사케     
18 맥문갱 여사 경       
19 선장    
20 백합 식물

여러분들이 좋아하는 가을의 특유의 분위기와 관련된 다른 단어가 있으면 추가해 주세요!
